In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb
from sklearn.metrics import mean_squared_error
color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
file_path = '/content/drive/My Drive/Energy/data/all_data/all_gas.xlsx'

df = pd.read_excel(file_path)

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
df

,MAINCD,MAINCDNM,PLAT,NEWPLAT,SIGUNGUCD,BJDONGCD,USEYM,USEQTY,ARCHAREA,TOAREA,REGPK,CLASSIFICATION,GAS_CHARGE,CARBON_EMSSIONS
0,04000,제2종근린생활시설,대전광역시 대덕구 신탄진동 155-19번지,대전광역시 대덕구 신탄진동로24번길 4,30230,12600,201801,307,70.44,140.88,30230-100177136,일반,2.479562e+04,675.4
1,18000,창고시설,대전광역시 대덕구 신일동 1683번지,대전광역시 대덕구 신일동로17번길 20,30230,11400,201801,31497,26.95,23.77,30230-100175809,산업,2.217351e+06,69293.4
2,11000,노유자시설,대전광역시 대덕구 연축동 269번지,대전광역시 대덕구 신탄진로 101,30230,10400,201801,54538,13.50,13.50,30230-100177398,일반,4.228249e+06,119983.6
3,11000,노유자시설,대전광역시 대덕구 연축동 269번지,대전광역시 대덕구 신탄진로 101,30230,10400,201801,54538,323.14,912.81,30230-100176677,일반,4.228249e+06,119983.6
4,03000,제1종근린생활시설,대전광역시 대덕구 송촌동 450-4번지,대전광역시 대덕구 송촌북로36번길 18,30230,10700,201801,6926,428.62,753.26,30230-100176956,일반,5.378354e+05,15237.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192548,10000,교육연구시설,대전광역시 유성구 장동 100번지,대전광역시 유성구 가정로 141,30200,12800,202307,212861,35.75,35.75,30200-14531,일반,1.649989e+07,468294.2
192549,10000,교육연구시설,대전광역시 유성구 장동 100번지,대전광역시 유성구 가정로 141,30200,12800,202307,212861,1364.55,2540.99,30200-14547,일반,1.649989e+07,468294.2
192550,10000,교육연구시설,대전광역시 유성구 장동 171번지,대전광역시 유성구 가정북로 156,30200,12800,202307,208781,1343.40,3375.40,30200-14593,일반,1.618365e+07,459318.2
192551,10000,교육연구시설,대전광역시 유성구 장동 171번지,대전광역시 유성구 가정북로 156,30200,12800,202307,208781,86.24,68.63,30200-14607,일반,1.618365e+07,459318.2


In [ ]:
df2 = df[['USEYM', 'REGPK', 'USEQTY']]
df2

,USEYM,REGPK,USEQTY
0,201801,30230-100177136,307
1,201801,30230-100175809,31497
2,201801,30230-100177398,54538
3,201801,30230-100176677,54538
4,201801,30230-100176956,6926
...,...,...,...
192548,202307,30200-14531,212861
192549,202307,30200-14547,212861
192550,202307,30200-14593,208781
192551,202307,30200-14607,208781


In [ ]:
# USEYM 열을 년월 형식으로 변환
df2['USEYM'] = pd.to_datetime(df2['USEYM'], format='%Y%m')

# 년월을 인덱스로 설정
df2.set_index('USEYM', inplace=True)

# 데이터프레임 확인
df2

<ipython-input-79-e9f5b01e085a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['USEYM'] = pd.to_datetime(df2['USEYM'], format='%Y%m')


,REGPK,USEQTY
USEYM,,
2018-01-01,30230-100177136,307
2018-01-01,30230-100175809,31497
2018-01-01,30230-100177398,54538
2018-01-01,30230-100176677,54538
2018-01-01,30230-100176956,6926
...,...,...
2023-07-01,30200-14531,212861
2023-07-01,30200-14547,212861
2023-07-01,30200-14593,208781


In [ ]:
import pandas as pd


df2['year'] = df2.index.year
df2['month'] = df2.index.month
df2['quarter'] = df2.index.quarter
df2['month_str'] = df2.index.strftime('%Y-%m')

df2

<ipython-input-80-f91b7b42ec54>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['year'] = df2.index.year
<ipython-input-80-f91b7b42ec54>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['month'] = df2.index.month
<ipython-input-80-f91b7b42ec54>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

,REGPK,USEQTY,year,month,quarter,month_str
USEYM,,,,,,
2018-01-01,30230-100177136,307,2018,1,1,2018-01
2018-01-01,30230-100175809,31497,2018,1,1,2018-01
2018-01-01,30230-100177398,54538,2018,1,1,2018-01
2018-01-01,30230-100176677,54538,2018,1,1,2018-01
2018-01-01,30230-100176956,6926,2018,1,1,2018-01
...,...,...,...,...,...,...
2023-07-01,30200-14531,212861,2023,7,3,2023-07
2023-07-01,30200-14547,212861,2023,7,3,2023-07
2023-07-01,30200-14593,208781,2023,7,3,2023-07


# 최종
# 20개랜덤

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

# 20개의 랜덤 REGPK를 선택합니다.
random_regpk = np.random.choice(df2['REGPK'].unique(), 20, replace=False)

# 예측 결과를 저장할 빈 데이터프레임을 생성합니다.
predictions_df = pd.DataFrame()

# 각 REGPK에 대한 모델링 및 예측을 수행합니다.
for regpk in random_regpk:
    # 특정 REGPK의 데이터를 선택합니다.
    building_data = df2[df2['REGPK'] == regpk]

    # 2018년 1월부터 2022년 12월까지의 데이터를 훈련 데이터로 사용합니다.
    train_data = building_data.loc[(building_data.index >= '2018-01') & (building_data.index <= '2022-12')]

    # XGBoost 모델을 초기화하고 훈련합니다.
    model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
    model.fit(train_data[['year', 'month', 'quarter']], train_data['USEQTY'])

    # 2023년 8월부터 12월까지 예측할 데이터 포인트를 생성합니다.
    future_dates = pd.date_range(start='2023-08-01', end='2023-12-31', freq='MS')
    future_data = pd.DataFrame(index=future_dates)
    future_data['year'] = future_data.index.year
    future_data['month'] = future_data.index.month
    future_data['quarter'] = future_data.index.quarter

    # 예측을 수행합니다.
    predictions = model.predict(future_data[['year', 'month', 'quarter']])
    future_data['prediction'] = predictions
    future_data['REGPK'] = regpk

    # 각 건물의 예측 결과를 predictions_df에 추가합니다.
    predictions_df = pd.concat([predictions_df, future_data.reset_index()])

# predictions_df 데이터프레임에 예측 결과가 저장되어 있습니다.
predictions_df = predictions_df[['REGPK', 'year', 'month', 'quarter', 'prediction']]
print(predictions_df)


              REGPK  year  month  quarter    prediction
0        30140-4327  2023      8        3  12881.417969
1        30140-4327  2023      9        3   9175.693359
2        30140-4327  2023     10        4   6459.180176
3        30140-4327  2023     11        4   5945.594727
4        30140-4327  2023     12        4   7470.705566
..              ...   ...    ...      ...           ...
0   30200-100270297  2023      8        3  11276.827148
1   30200-100270297  2023      9        3  10015.946289
2   30200-100270297  2023     10        4   8454.147461
3   30200-100270297  2023     11        4   7606.097168
4   30200-100270297  2023     12        4  10530.406250

[100 rows x 5 columns]


# 전체

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

# 모든 고유 REGPK를 선택합니다.
all_regpk = df2['REGPK'].unique()

# 예측 결과를 저장할 빈 데이터프레임을 생성합니다.
predictions_df = pd.DataFrame()

# 각 REGPK에 대한 모델링 및 예측을 수행합니다.
for regpk in all_regpk:
    # 특정 REGPK의 데이터를 선택합니다.
    building_data = df2[df2['REGPK'] == regpk]

    # 2018년 1월부터 2022년 12월까지의 데이터를 훈련 데이터로 사용합니다.
    train_data = building_data.loc[(building_data.index >= '2018-01') & (building_data.index <= '2022-12')]

    # XGBoost 모델을 초기화하고 훈련합니다.
    model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
    model.fit(train_data[['year', 'month', 'quarter']], train_data['USEQTY'])

    # 2023년 8월부터 12월까지 예측할 데이터 포인트를 생성합니다.
    future_dates = pd.date_range(start='2023-08-01', end='2023-12-31', freq='MS')
    future_data = pd.DataFrame(index=future_dates)
    future_data['year'] = future_data.index.year
    future_data['month'] = future_data.index.month
    future_data['quarter'] = future_data.index.quarter

    # 예측을 수행합니다.
    predictions = model.predict(future_data[['year', 'month', 'quarter']])
    future_data['prediction'] = predictions
    future_data['REGPK'] = regpk

    # 각 건물의 예측 결과를 predictions_df에 추가합니다.
    predictions_df = pd.concat([predictions_df, future_data.reset_index()])

# predictions_df 데이터프레임에 예측 결과가 저장되어 있습니다.
predictions_df = predictions_df[['REGPK', 'year', 'month', 'quarter', 'prediction']]
print(predictions_df)


              REGPK  year  month  quarter  prediction
0   30230-100177136  2023      8        3   12.131174
1   30230-100177136  2023      9        3   12.229965
2   30230-100177136  2023     10        4   23.236700
3   30230-100177136  2023     11        4   13.962869
4   30230-100177136  2023     12        4   12.750124
..              ...   ...    ...      ...         ...
0       30170-13665  2023      8        3    0.000000
1       30170-13665  2023      9        3    0.000000
2       30170-13665  2023     10        4    0.000000
3       30170-13665  2023     11        4    0.000000
4       30170-13665  2023     12        4    0.000000

[16060 rows x 5 columns]


In [ ]:
predictions_df

,REGPK,year,month,quarter,prediction
0,30230-100177136,2023,8,3,12.131174
1,30230-100177136,2023,9,3,12.229965
2,30230-100177136,2023,10,4,23.236700
3,30230-100177136,2023,11,4,13.962869
4,30230-100177136,2023,12,4,12.750124
...,...,...,...,...,...
0,30170-13665,2023,8,3,0.000000
1,30170-13665,2023,9,3,0.000000
2,30170-13665,2023,10,4,0.000000
3,30170-13665,2023,11,4,0.000000


ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ

In [ ]:
predictions_df['USEYM'] = predictions_df['year'].astype(str) + predictions_df['month'].astype(str).str.zfill(2)
predictions_df

,REGPK,year,month,quarter,prediction,USEYM
0,30230-100177136,2023,8,3,12.131174,202308
1,30230-100177136,2023,9,3,12.229965,202309
2,30230-100177136,2023,10,4,23.236700,202310
3,30230-100177136,2023,11,4,13.962869,202311
4,30230-100177136,2023,12,4,12.750124,202312
...,...,...,...,...,...,...
0,30170-13665,2023,8,3,0.000000,202308
1,30170-13665,2023,9,3,0.000000,202309
2,30170-13665,2023,10,4,0.000000,202310
3,30170-13665,2023,11,4,0.000000,202311


In [ ]:
predictions_df.rename(columns={'prediction': 'USEQTY'}, inplace=True)
predictions_df

,REGPK,year,month,quarter,USEQTY,USEYM
0,30230-100177136,2023,8,3,12.131174,202308
1,30230-100177136,2023,9,3,12.229965,202309
2,30230-100177136,2023,10,4,23.236700,202310
3,30230-100177136,2023,11,4,13.962869,202311
4,30230-100177136,2023,12,4,12.750124,202312
...,...,...,...,...,...,...
0,30170-13665,2023,8,3,0.000000,202308
1,30170-13665,2023,9,3,0.000000,202309
2,30170-13665,2023,10,4,0.000000,202310
3,30170-13665,2023,11,4,0.000000,202311


In [ ]:
predictions_df = predictions_df[['REGPK', 'USEQTY', 'USEYM']]
predictions_df.to_csv("/content/drive/My Drive/Energy/pred/PRED_REGPK_GAS.csv")